In [1]:
# youtube8m のディレクトリに移動
import os
while os.getcwd().split('/')[-1] != 'youtube8m': os.chdir('..')
os.getcwd()

'/home/jupyter/ASLOpenProject/youtube8m'

In [2]:
import tensorflow as tf
from video.data import read_dataset

In [3]:
tf.enable_eager_execution()

# カテゴリ一覧

In [4]:
!gsutil cp gs://asl-mixi-project-bucket/data/youtube-8m/vocabulary.csv .

Copying gs://asl-mixi-project-bucket/data/youtube-8m/vocabulary.csv...
/ [1 files][  3.4 MiB/  3.4 MiB]                                                
Operation completed over 1 objects/3.4 MiB.                                      


In [5]:
import csv

label_map = {}
with open('./vocabulary.csv', 'r') as c:
    reader = csv.reader(c)
    next(reader, None)
    for r in reader:
        label_map[r[0]] = r[3]

# 評価データセットの読み込み

In [6]:
data = read_dataset(
    'gs://asl-mixi-project-bucket/data/youtube-8m/valid/*.tfrecord',
    tf.estimator.ModeKeys.EVAL,
    batch_size=5
).shuffle(1024)

# 推論

In [7]:
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials
import json

PROJECT = 'qwiklabs-gcp-ml-83914069970b'
MODEL_NAME = 'youtube8mkojo'
VERSION = 'v1'

credentials = GoogleCredentials.get_application_default()
api = discovery.build(
    "ml",
    "v1",
    credentials = credentials,
    discoveryServiceUrl = "https://storage.googleapis.com/cloud-ml/discovery/ml_v1_discovery.json"
)

def convert_data(data):
    batch, labels = next(iter(data))
    batch.pop('id')
    feature = [{'mean_rgb': r.numpy().tolist(), 'mean_audio': a.numpy().tolist()} for r, a in zip(batch['mean_rgb'], batch['mean_audio'])]
    labels = [[label_map[str(i)] for i, v in enumerate(label) if v] for label in labels]
    return feature, labels

def show_predict(prediction, label):
    print('labels:', label)
    for i, index in enumerate(prediction["predicted_topk"]):
        print("{}th({}:{}) probality is {}".format(
            (i+1),
            index,
            label_map["{}".format(index)],
            prediction['probabilities'][index]
        ))
    print('')

def predict(data, model_name=MODEL_NAME, version=VERSION):
    feature, labels = convert_data(data)
    request_data = {"instances": feature}
    parent = "projects/{}/models/{}/versions/{}".format(PROJECT,MODEL_NAME, VERSION)
    response = api.projects().predict(body = request_data, name = parent).execute()
    for p, l in zip(response['predictions'], labels):
        show_predict(p, l)
    return None

In [8]:
predict(data, 'youtube8mkojo', 'v1')

labels: ['Game', 'Cartoon']
1th(5:Cartoon) probality is 0.9971925020217896
2th(0:Game) probality is 0.991986870765686
3th(769:Penguin) probality is 0.487371563911438
4th(1:Video game) probality is 0.4732784032821655
5th(1183:Monkey) probality is 0.026076853275299072

labels: ['Dentist']
1th(1493:Dentist) probality is 0.85809725522995
2th(674:Skin) probality is 0.06330987811088562
3th(579:Medicine) probality is 0.03888842463493347
4th(2647:Birth) probality is 0.030201762914657593
5th(866:Face) probality is 0.025571435689926147

labels: ['Game', 'Guild Wars']
1th(0:Game) probality is 0.8760881423950195
2th(591:Guild Wars) probality is 0.8667747974395752
3th(1:Video game) probality is 0.13755053281784058
4th(262:Farm) probality is 0.05680587887763977
5th(856:Silkroad Online) probality is 0.014575213193893433

labels: ['Cartoon', 'Sailor Moon', 'Sailor Moon (character)']
1th(5:Cartoon) probality is 0.9866054058074951
2th(597:Sailor Moon) probality is 0.8544175028800964
3th(1030:Sailor Moon